# Implementation and Testing

## Situation Workflow For Testing

```mermaid
sequenceDiagram
participant Sam
participant HeyRoommate
participant Nina

Sam->>HeyRoommate: Opens app and selects “Bill Reminder” topic
HeyRoommate->>HeyRoommate: Identifies household + active roommate profiles
HeyRoommate->>HeyRoommate: Loads Sam's communication style + conflict history

HeyRoommate->>Sam: Suggests relevant past scenarios (money/expenses)
HeyRoommate->>Sam: Displays insights (best tone: neutral, avoid urgency language)

Sam->>HeyRoommate: Drafts raw message (emotional or direct)
Sam->>HeyRoommate: Adjusts tone slider (“neutral + efficient”)
Sam->>HeyRoommate: Confirms recipient (Nina)

HeyRoommate->>HeyRoommate: Analyzes message sentiment + intent
HeyRoommate->>HeyRoommate: Maps message to Nina’s tone preference (warm + brief)
HeyRoommate->>HeyRoommate: Flags potential friction words

HeyRoommate->>Sam: Generates 3 reworded message options
HeyRoommate->>Sam: Shows rationale for each version
HeyRoommate->>Sam: Displays confidence score + predicted response likelihood

Sam->>HeyRoommate: Selects preferred message option
Sam->>HeyRoommate: Sends message via in-app DM

HeyRoommate->>Nina: Delivers message adapted to Nina’s tone profile
HeyRoommate->>Nina: Adds subtle context tag (“Friendly reminder”)

Nina->>HeyRoommate: Reads message
Nina->>HeyRoommate: Replies with acknowledgment + payment confirmation

HeyRoommate->>HeyRoommate: Analyzes response speed + sentiment
HeyRoommate->>HeyRoommate: Updates tone effectiveness metrics

HeyRoommate->>Sam: Summarizes outcome (resolved, low tension)
HeyRoommate->>Sam: Stores interaction as successful reference


## Implementation of Situation

In [5]:
from openai import OpenAI
import os
import base64
import requests

from dotenv import load_dotenv
# Load API key
_ = load_dotenv()

In [6]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [7]:
client = OpenAI()

In [8]:
system_message = {
    "role": "system",
    "content": (
        "You are HeyRoommate, an AI trained to transform tense, emotional, or delicate "
        "messages between roommates into clear, neutral, and effective communication. "
        "You must never escalate conflict or use accusatory, passive-aggressive, "
        "guilt-trippy, or shaming language."
    )
}


Input = {
    "role": "user",
    "content": """
We are running STEP 1 of the 'Bill Reminder' workflow.

Your job in this step is ONLY to ask me the following questions,
ONE AT A TIME, waiting for my answer each time.

Ask in this order:

1. "What draft message do you want to send about the bill?"

2. "Which specific bill is this (e.g., electric, WiFi, groceries)?"

3. Ask for a WARMTH slider for Sam's tone:
   "On a scale from 1 to 5, how WARM should your message feel?
    (1 = very cool/neutral, 3 = balanced, 5 = very warm/supportive)."

4. Ask for a LENGTH/DIRECTNESS slider for Sam's tone:
   "On a scale from 1 to 5, how LONG vs. BRIEF should your message be?
    (1 = very brief/direct, 3 = medium, 5 = more detailed/softened)."

5. Confirm Nina's tone profile (preloaded from her profile and past analysis):
   "Nina's current tone profile is 'warm + brief' based on her past messages.
    Does that still feel accurate to you? (yes/no, or suggest a small adjustment)."

6. "Has there been any past tension or context around money or bills that I should know?"

7. "What outcome do you want from this message (e.g., payment today, acknowledgment, gentle reminder)?"

Rules:
- Ask exactly ONE question per response.
- Do NOT rewrite any messages.
- Do NOT summarize.
- After you have asked Question 7, append the text 'DONE ASKING QUESTIONS' at the end of your last message.
"""
}

messages = [system_message, Input]

while True:
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    assistant_reply = completion.choices[0].message.content
    print("\nASSISTANT:", assistant_reply)

    messages.append({"role": "assistant", "content": assistant_reply})

    # Break after the model says it's done
    if "DONE ASKING QUESTIONS" in assistant_reply:
        break

    user_answer = input("\nYOUR ANSWER: ")
    messages.append({"role": "user", "content": user_answer})


ASSISTANT: What draft message do you want to send about the bill?



YOUR ANSWER:  please pay



ASSISTANT: Which specific bill is this (e.g., electric, WiFi, groceries)?



YOUR ANSWER:  electric



ASSISTANT: On a scale from 1 to 5, how WARM should your message feel? (1 = very cool/neutral, 3 = balanced, 5 = very warm/supportive).


...


ASSISTANT: Nina's current tone profile is 'warm + brief' based on her past messages. Does that still feel accurate to you? (yes/no, or suggest a small adjustment).



YOUR ANSWER:  yes



ASSISTANT: Has there been any past tension or context around money or bills that I should know?



YOUR ANSWER:  no



ASSISTANT: What outcome do you want from this message (e.g., payment today, acknowledgment, gentle reminder)?  

DONE ASKING QUESTIONS


In [9]:
Creation = {
    "role": "user",
    "content": """
Now run the 'Bill Reminder' rewrite step.

Using all of the information above (my answers to your questions), do the following:

1. Treat this as a 'Bill Reminder' topic with past outcomes tagged 'money/expenses'
   that suggest softer, brief reminders usually work best with Nina.

2. Map my slider responses to a tone for Sam:
   - WARMTH: 1–5 scale (1 = cool/neutral, 5 = very warm/supportive)
   - LENGTH: 1–5 scale (1 = very brief/direct, 5 = more detailed/softened)

3. Use Nina's confirmed tone profile with any small adjustments I gave
   as the target tone for the delivered message.

4. Include any numbers for bill price mentioned in the inputs. Be sure to include any and all content from the user input message in nina's desired tone.

5. Generate exactly 3 candidate rewrites of Sam's message.

For EACH option, include the following fields in plain text (no JSON, no lists):

Option 1:
Message: "..."
Tone match score (to my slider settings + Nina profile): X/10
Explanation: 1–2 sentences describing tone-fit only.

Option 2:
Message: "..."
Tone match score: X/10
Explanation: ...

Option 3:
Message: "..."
Tone match score: X/10
Explanation: ...

Constraints:
- Message must clearly reference the bill.
- Tone must be respectful and non-accusatory.
- No guilt-tripping, sarcasm, or shaming.
- Explanations should discuss tone-fit ONLY.

Do not ask me any additional questions.
Just present the three options exactly as described.
"""
}

messages_phase2 = messages + [Creation]

completion2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages_phase2
)

options_text = completion2.choices[0].message.content

print("\n---- REWRITE OPTIONS ----\n")
print(options_text)

option_choice = input("\nWhich option would you like to select? Enter 1, 2, or 3: ")


---- REWRITE OPTIONS ----

Option 1:
Message: "Can you pay the electric bill today, please?"
Tone match score: 8/10
Explanation: This message is very brief and direct, matching your preference for a cool, concise tone while still maintaining respect.

Option 2:
Message: "Reminder: We need the electric bill settled soon. Could you handle it today?"
Tone match score: 7/10
Explanation: This option is slightly more framed as a request rather than a direct statement, providing a bit more warmth while ensuring clarity.

Option 3:
Message: "Please take care of the electric bill today. Thanks!"
Tone match score: 9/10
Explanation: This is brief and to the point with a touch of politeness, aligning closely with your slider settings and Nina's tone profile.



Which option would you like to select? Enter 1, 2, or 3:  3


In [10]:
Simulation = {
    "role": "user",
    "content": f"""
I have now chosen Option {option_choice} from the three options you presented.

Using the same context as before, and Nina's tones profile ('warm + brief'), do the following:

1. Restate Option {option_choice}'s message in your own words so it is clear which one is being sent.

2. Simulate Nina's likely response, assuming:
   - Her reply changes depending on what the emssage we sent was
   - She is most receptive to responds the type of messages you have been told she responds best to
   - She responds more favorably to texts that are more aligned with her preferences (i.e. she is more likely to send the payment)
   - She will respond more warmly to warmer messages, unless she has strong vause to be apologetic

3. Provide a short outcome summary for Sam, including:
   - Whether the goal (payment + low tension) was achieved.
   - How well Option {option_choice} aligned with:
       (a) Sam's warmth + length slider settings
       (b) Nina's 'warm + brief' tone profile.
   - A single-sentence 'tone effectiveness note' for future reminders
     (e.g., "Warm openers + concise reminders continue to work well for Nina on money topics.").

Do NOT ask any further questions.
Do NOT generate new options.
Respond only with:
- Nina's simulated reply
- Outcome summary
- Tone-effectiveness update
"""
}

simulation_messages = messages_phase2 + [
    {"role": "assistant", "content": options_text},
    {"role": "user", "content": f"I choose option {option_choice}."},
    Simulation
]

completion3 = client.chat.completions.create(
    model="gpt-4o",
    messages=simulation_messages
)

print("\n---- SIMULATION RESULT ----\n")
print(completion3.choices[0].message.content)


---- SIMULATION RESULT ----

Restated Message: "Please take care of the electric bill today. Thanks!"

Nina's Simulated Reply: "Sure, I'll get it done today. Thanks for the reminder!"

Outcome Summary:
- The goal of securing payment with low tension was achieved successfully.
- Option 3 aligned very well with both (a) Sam's slider settings for a cool, brief message; and (b) Nina's preference for a warm and brief tone.
- Tone-effectiveness note: "Concise and polite requests continue to work effectively with Nina for bill-related messages."
